In [ ]:

# leba3207
from unicodedata import category
from tqdm import tqdm
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import RandomForestRegressor

import difflib
from functools import partial

Chargement des données des différents fichiers

In [ ]:

data_folder = 'data/'

journal_file = 'api_journal11-13-17.csv'
price_file = 'api_price11-13-17.csv'
influence_file = 'estimated-article-influence-scores-2015.csv'

journal = pd.read_csv(data_folder + journal_file, sep=',', encoding='latin1')
price = pd.read_csv(data_folder + price_file, sep=',', index_col=0)
influence = pd.read_csv(data_folder + influence_file, sep=',', index_col=0)


In [ ]:

def get_uniqueness_attributes(table):
    return table.nunique(axis=0)


def get_ratio_missing_values(table):
    return table.isnull().sum() * 100 / len(table)


def get_unique_values_of_attribute(table, header):
    return table[header].unique()


def lowercase_columns(table, headers):
    for header in headers:
        table[header] = table[header].str.lower()


# TODO: include headers specification
def get_df_duplicated_rows_dropped(table):
    return table.drop_duplicates()


def plot_categories_frequency(table, header):
    fig, ax = plt.subplots()
    table[header].value_counts()[0:5].plot(ax=ax, kind='bar')
    plt.title(f'Fréquence d\'apparition des catégories de l\'attribut {header}')
    plt.show()


def get_mean_price_per_year():
    mean_price_per_year = {}
    for index, p in price.iterrows():
        year = p['date_stamp'].year
        if year in mean_price_per_year:
            mean_price_per_year[year] += p['price']
        else:
            mean_price_per_year[year] = p['price']

    for year, value in mean_price_per_year.items():
        mean_price_per_year[year] /= len(price[price['price'] != 0])
    return {k: v for k, v in sorted(mean_price_per_year.items(), key=lambda item: item[1], reverse=True)}


def rename_df_headers(table, dict_headers):
    return table.rename(columns=dict_headers)


def get_score_sequence_matching(s, c1, category):
    if s[c1] is np.nan:
        return 0
    return difflib.SequenceMatcher(None, s[c1], category).ratio()


def get_empty_attribute_to_remove(table):
    headers = []
    for header in table.columns:
        if table[header].isna().sum() * 100 / len(table) > 50:
            headers.append(header)
    return headers


# Question 1: Exploration-Description
## Présenter une description de chacun des attributs des 3 tables, avec des graphiques pour la visualisation des 
statistiques descriptives au besoin.

### Table journal

In [ ]:
# TODO: visualisations
# fréquence des valeurs

In [ ]:

print(journal.head())

In [ ]:

"""
issn: identifiant du journal
Les valeurs de cet attribut semblent suivre un format particulier tel que: 4 digits - 4 digits

journal_name: nom textuel du journal
Les valeurs sont textuelles, ne suivant pas de valeurs catégorielles particulière à priori.

pub_name: nom de l'éditeur du journal
Les valeurs sont textuelles, ne suivant pas de valeurs catégorielles particulière à priori.

is_hybrid: indique si le journal est hybride, ce qui signifie que c'est une revue sur abonnement dont certains articles
sont en libre accès, comme l'indique le site http://flourishoa.org/about.

category: renseigne sur la/les catégorie(s) des sujets abordés par le journal 
Les valeurs sont textuelles et sont catégorielles. Chaque objet peut posséder des valeurs multiples pour cet attribut. La séparation entre les différentes valeurs semblent
être inconsistante.

url: indique l'adresse url du site du journal


Les attributs journal_name, pub_name et category étant des données textuelles très inconsistantes, je décide avant tout
traitement et étude supplémentaire de transformer les valeurs en minuscule pour limiter au maximum l'inconsistence.
"""

In [ ]:

lowercase_columns(journal, ['journal_name', 'pub_name', 'category'])

In [ ]:

print(f"Valeurs uniques des attributs de journal:\n"
      f"{get_uniqueness_attributes(journal)}")
print(f"Ratio de valeurs vides pour les attributs de journal:\n"
      f"{get_ratio_missing_values(journal)}")

print(f"Valeurs possibles pour l'attribut is_hybrid de journal:\n"
      f"{get_unique_values_of_attribute(journal, 'is_hybrid')}")
print(f"Valeurs possibles pour l'attribut category de journal:\n"
      f"{get_unique_values_of_attribute(journal, 'category')}")

Les attributs category et url présentent un nombre conséquent de valeurs manquantes.
L'attribut issn présente des valeurs uniques pour chacun de ses objets.

On remarque qu'il existe uniquement deux valeurs pour l'attribut is_hybrid (soit 1 soit 0). 

In [ ]:

print(f"Valeurs possibles de pub_name quand is_hybrid vaut 1:\n"
      f"{journal[journal['is_hybrid'] == 1]['pub_name'].unique()}")

In [ ]:

plot_categories_frequency(journal, 'pub_name')
plot_categories_frequency(journal, 'category')

### Table price

In [ ]:

print(price.head())

price: information du prix d'une publication pour le journal associé à une date précise, en dollar US
Si celui-ci est à 0, on peut coompendre que celui-ci est gratuit

date_stamp: horodatage de l'information de prix d'une publication, en format années-mois-jour

journal_id: identifiant du journal
Les valeurs semblent suivre consistantement un format du type: 4 digits - 4 digits

influence_id: identifiant de l'influence
Les valeurs suivent un format 4 digits.

url: indique l'adresse url du site de l'auteur

license: indique le type de license utilisé par le journal pour les différents articles utilisés.


On convertit l'attribut date_stamp en type date.

In [ ]:

price['date_stamp'] = pd.to_datetime(price['date_stamp'], errors='coerce', format='%Y-%m-%d')

In [ ]:

print(f"Valeurs uniques des attributs de price:\n"
      f"{get_uniqueness_attributes(price)}")
print(f"Ratio de valeurs vides pour les attributs de price:\n"
      f"{get_ratio_missing_values(price)}")

print(f"Exemples de valeurs possibles pour l'attribut influence_id de price:\n"
      f"{get_unique_values_of_attribute(price, 'influence_id')[1:8]}")
print(f"Valeurs possibles pour l'attribut license de price:\n"
      f"{get_unique_values_of_attribute(price, 'license')}")

Les attributs influence_id, url et license présentent une majorité de valeurs manquantes.

In [ ]:

mean_price_per_year = get_mean_price_per_year()

plt.bar(range(len(mean_price_per_year)), mean_price_per_year.values())
plt.xticks(range(len(mean_price_per_year)), mean_price_per_year.keys())
plt.title("Moyenne par année des prix des publications")
plt.show()

### Table influence

In [ ]:

print(influence.head())

# TODO: proj_ai moyenne

journal_name: nom textuel du journal
Les valeurs sont textuelles, ne suivant pas de valeurs catégorielles particulière à priori.

issn: identifiant du journal
Les valeurs de cet attribut semblent suivre un format particulier tel que: 4 digits - 4 digits

citation_count_sum: indique le nombre de citations du jounal

paper_count_sum: indique le nombre de citations des articles du jounal

avg_cites_per_paper: indique la moyenne des citations par papier qui sont contenus du journal

proj_ai: information sur le score d'influence des articles du journal

proj_ai_year: spécification de l'année où l'information sur le score d'influence des articles du journal a été établie

In [ ]:

print(f"Valeurs uniques des attributs de influence:\n"
      f"{get_uniqueness_attributes(influence)}")
print(f"Ratio de valeurs vides pour les attributs de influence:\n"
      f"{get_ratio_missing_values(influence)}")

print(f"Valeurs possibles pour l'attribut proj_ai_year de influence:\n"
      f"{get_unique_values_of_attribute(influence, 'proj_ai_year')}")

L'attribut proj_ai_year ne présentant qu'une seule valeur nous indique que les valeurs de l'attribut proj_ai ont toutes
été établies à la même période. 
"""

# TODO: proj_ai sum in function of journal

# Question 2: Prétraitement-Représentation
## A. Effectuer un prétraitement des données pour supprimer les duplications et corriger les incohérences s’il y en a.

### Table journal
Dans un premier temps, on élimine les objets présentant des objets dupliqués sur tous les attributs.
On se base sur l'attribut issn qui devrait être unique pour chaque objet de la table, on vérifie son unicité.

In [ ]:

nb = len(journal)
journal = get_df_duplicated_rows_dropped(journal)
print(f"Nombre d'objets dupliqués éliminés dans journal: {nb - len(journal)}")

check = np.logical_not(journal['issn'].duplicated().any())
print(f"Unicité de l'attribut issn dans la table journal: {check}")

### Table price
Etant donné que les index étaient fournis dans le fichier original et qu'on les utilise afin d'indexer nos objets, 
on vérifie qu'il n'existe pas de duplicata.

Ensuite, on élimine les objets présentant des objets dupliqués sur tous les attributs.
Dans un second temps, dans la table price, les objets se doivent d'être uniques selon deux attributs, date_stamp et
journal_id. S'ils ne le sont pas, alors ceux sont des objets dupliqués.

In [ ]:

check = np.logical_not(price.index.duplicated().any())
print(f"Unicité des indexes de la table price: {check}")

nb = len(price)
price = get_df_duplicated_rows_dropped(price)
print(f"Nombre d'objets dupliqués éliminés dans price: {nb - len(price)}")

duplicated_rows = price[price[['date_stamp', 'journal_id']].duplicated(keep=False)]

Il existe des duplicata ambigus que l'on décide de traiter un à un.

Premier cas: un des objets présente un prix nul, on décide de choisir de l'éliminer au profit de l'autre.

In [ ]:

print(duplicated_rows.iloc[0].fillna(0) == duplicated_rows.iloc[1].fillna(0))
price = price.drop(duplicated_rows.index.values[0])

Deuxième cas: leur valeur du prix est différente d'un léger écart, on décide de garder la deuxième de manière
arbitraire

In [ ]:

print(duplicated_rows.iloc[40].fillna(0) == duplicated_rows.iloc[41].fillna(0))
price = price.drop(duplicated_rows.index.values[40])

Troisième cas: seule la valeur de license est différente, on décide de garder la première de manière arbitraire.

In [ ]:

for i in range(3, 39, 2):
    price = price.drop(duplicated_rows.index.values[i])

### Table influence
Etant donné que les index étaient fournis dans le fichier original et qu'on les utilise afin d'indexer nos objets, 
on vérifie qu'il n'existe pas de duplicata.

On se base sur l'attribut issn qui devrait être unique pour chaque objet de la table, on vérifie son unicité.

In [ ]:

nb = len(influence)
influence = get_df_duplicated_rows_dropped(influence)
print(f"Nombre d'objets dupliqués éliminés dans influence: {nb - len(influence)}")

check = np.logical_not(influence['issn'].duplicated().any())
print(f"Unicité de l'attribut issn dans la table influence: {check}")

## Merge
Afin de simplifier les opérations, on génère une seule table reprenant les informations des trois tables.
On vérifie d'abord si les identifiants communs aux différentes tables sont présentes dans les tables à merger.
En premier, on vérifie si les valeurs de l'attribut issn de influence sont existantes dans l'attribut du même nom 
dans journal.
De même, on vérifie les valeurs de l'attribut journal_id de price sont existantes dans l'attribut issn dans journal. 

In [ ]:

check = influence['issn'].isin(journal['issn']).any()
print(f"Pas de valeur d'issn manquante dans journal par rapport à influence : {check}")

check = price['journal_id'].isin(journal['issn']).any()
print(f"Pas de valeur d'issn manquante dans journal par rapport à price : {check}")

On applique maintenant le merge des trois tables en deux étapes. D'abord, on merge influence dans journal, puis price
est ensuite mergé dans le résultat du premier merge.

In [ ]:

price = rename_df_headers(price, {"journal_id": "issn", "url": "url_autor"})
journal = rename_df_headers(journal, {"url": "url_journal"})

temp = pd.merge(journal, influence, on='issn', how='outer')
check = len(temp[temp['journal_name_x'] != temp['journal_name_y']])
print(f"Nombre d'aberrances entre les valeurs journal_name des tables journal et influence: {check}")
temp = temp.drop(columns=['journal_name_y'])
temp = rename_df_headers(temp, {"journal_name_x": "journal_name"})

print(f"Valeurs uniques des attributs de temp:\n"
      f"{get_uniqueness_attributes(temp)}")

data = pd.merge(temp, price, on=['issn'], how='outer')
data = get_df_duplicated_rows_dropped(data)

print(f"Valeurs uniques des attributs de data:\n"
      f"{get_uniqueness_attributes(data)}")
print(f"Ratio de valeurs vides pour les attributs de data:\n"
      f"{get_ratio_missing_values(data)}")

On s'assure bien que les valeurs de l'attribut issn de la nouvelle date (data) sont uniques.

## B. Y-a-t-il une corrélation entre les catégories de journaux (attribut category) et les coûts de publication 
(attribut price)? Justifier la réponse.

Afin de déterminer s'il existe une corrélation entre les catégories et l'attribut prix, on s'intéresse à chaque 
catégorie une à une. 
Etant donné que chaque objet peut avoir plusieurs valeurs de catégories, on décide de séparer les catégories selon les 
différents séparateurs observés (|, and, .). On les convertit ensuite en one hot.
On calcule ensuite la corrélation catégorie par catégorie avec l'attribut prix. Pour cela, on ne considère que les 
objets présentant la catégorie testée et les valeurs de prix associées.

In [ ]:

cat_labelled_data = data[data['category'].notna()]
cat_data_to_predict = data[data['category'].isna()]

In [ ]:

cat_labelled_data['category'] = cat_labelled_data['category'].str.replace(r'[\.\|&] | [\.\|&] | and ', '.', regex=True)
category_dummies = cat_labelled_data['category'].str.get_dummies(sep='.')
category_dummies_prefix = category_dummies.add_prefix('category_')
print(f'Nombre de catégories après séparation: {category_dummies.shape[1]}')
# %%

cat_labelled_data = pd.concat([cat_labelled_data, category_dummies_prefix], axis=1) \
    # .drop(columns=['category'])

In [ ]:

categories_correlation = {}

for header in category_dummies_prefix.columns:
    corr = cat_labelled_data[header].corr(cat_labelled_data['price'])
    if abs(corr) > 0.1:
        categories_correlation[header] = corr

fig, ax = plt.subplots()
plt.bar(categories_correlation.keys(), categories_correlation.values())
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.title(f'Catégories présentant des corrélations fortes avec\nl\'attribut prix et leurs valeurs')
plt.show()

On remarque que certaines catégories présentent effectement une légère corrélation avec l'attribut prix.
(Les catégories présentant une corrélation inférieures à 0.1 ne sont pas incluses dans le graphe)
Les catégories présentant la plus forte corrélation sont 'cell biology' et 'molecular'.
Cependant, cette corrélation remarquée est très faible et peut être négligeable.

## C. Construire un modèle pour prédire les valeurs de catégorie de journaux manquantes de la façon la plus précise
possible (cela inclut la sélection d’attributs informatifs, le choix et le paramétrage d’un modèle de classification,
le calcul du score du modèle, l’application du modèle pour prédire les catégories manquantes). Justifier les choix
effectués.
TODO: remove attributes price and citations from 2.C and ajust justification
Dans le but de prédire les catégories de journaux, on doit s'intéresser à plusieurs attributs qui pourraient nous
aider. Le nom du journal pourrait inclure certains mots-clés qui pourraient s'apparenter aux catégories du journal.
Le nom de l'éditeur pourrait également apporter de l'information sur les catégories.
Etant donné qu'on a pu trouver certaines corrélations entre l'attribut prix et les catégories, on prend également en
compte ce paramètre.
Les informations de citation du journal pourraient également se révéler porteuses d'informations, ainsi que l'influence
des articles.

In [ ]:

cat_labelled_data = cat_labelled_data[cat_labelled_data['price'].notna()]
headers = ['citation_count_sum', 'paper_count_sum', 'avg_cites_per_paper', 'proj_ai', 'price']
cat_labelled_data = cat_labelled_data.dropna(axis=0, subset=headers)

In [ ]:

for header in tqdm(category_dummies.columns):
    cat_labelled_data['jn_' + header] = cat_labelled_data.apply(partial(get_score_sequence_matching, c1='journal_name',
                                                                        category=header), axis=1)
    cat_labelled_data['pn_' + header] = cat_labelled_data.apply(partial(get_score_sequence_matching, c1='pub_name',
                                                                        category=header), axis=1)

In [ ]:

cat_labelled_data = cat_labelled_data.drop(columns=['category'])
print(f'size cat_labelled_data before splitting: {cat_labelled_data.shape[0]}')

In [ ]:

jn_sm_headers = cat_labelled_data.filter(like='jn_').columns.to_list()
pn_sm_headers = cat_labelled_data.filter(like='pn_').columns.to_list()
# TODO: add date_stamp to attributes_of_interest
attributes_of_interest = ['citation_count_sum', 'paper_count_sum', 'avg_cites_per_paper', 'proj_ai', 'price',
                          # 'date_stamp',
                          ]
attributes_of_interest.extend(jn_sm_headers)
attributes_of_interest.extend(pn_sm_headers)

### Entrainement
On applique des modèles de classification ayant la capacité de pouvoir préduire des labels multiples.
Pour cela, on utilise la méthode MultiOutputClassifier de sklearn afin qui consiste à adapter un classificateur par
cible.
A partir de là, on a pu essayer plusieurs types de classification, les deux meilleurs se sont révélés être les random
forest et les K plus proches voisins.

Le code suivant sert à faire une recherche d'hyperparamètres (succinte) sur un classification random forest.
Pour le confort du temps de compilation, je n'ai pas intégré au rendu le classification K plus proche voisin, cependant
le résultat du meilleur modèle trouvé est décrit ci-dessous.

In [ ]:

clfs = {'RandomForestClassifier': RandomForestClassifier()}

best_model = {'name': '', 'score': 0, 'model': None}
for name, clf in clfs.items():
    # for i in range(15, 17): # TODO
    for i in range(13, 14):
        X_train, X_test, y_train, y_test = train_test_split(cat_labelled_data[attributes_of_interest],
                                                            cat_labelled_data[category_dummies_prefix.columns],
                                                            test_size=0.33, random_state=42)
        X_train = X_train[attributes_of_interest]
        X_test = X_test[attributes_of_interest]
        y_train = y_train.to_numpy()

        clf.set_params(max_depth=i)

        print(f'Modèle {name} {i}')
        print('-- Entrainement')
        classifier = MultiOutputClassifier(clf, n_jobs=-1)
        classifier.fit(X_train, y_train)
        train_score = classifier.score(X_train, y_train)
        print(f'Score d\'entraînement: {train_score}')

        print('-- Test')
        test_predictions = classifier.predict(X_test)
        test_score = classifier.score(X_test, y_test)
        print(f'Score de test: {test_score}')

        if test_score > best_model.get('score'):
            best_model['name'], best_model['score'], best_model['model'] = name + ' ' + str(i), test_score, classifier

print(f"Le modèle présentant le meilleur score est {best_model.get('name')} avec {best_model.get('score')}")

Résultats des différents essais:

Modèle RandomForestClassifier 12
-- Entrainement
Score d'entraînement: 0.9916259595254711
-- Test
Score de test: 0.7312588401697313

Modèle RandomForestClassifier 13
-- Entrainement
Score d'entraînement: 0.994417306350314
-- Test
Score de test: 0.7369165487977369

On conclut ainsi que le classificateur random forest ayant une profondeur maximale de 13 présente des résultats se
trouve être le plus performant.
Aussi, on se trouve en présence de résultats très performants, étant donné qu'on dispose de 88 labels à prédire qui 
sont les catégories.

### Prédictions
On effectue maintenant les prédictions sur les objets présentant les catégories manquantes.

In [ ]:

cat_data_to_predict = cat_data_to_predict.dropna(axis=0, subset=headers)

In [ ]:

for header in tqdm(category_dummies.columns):
    cat_data_to_predict['jn_' + header] = cat_data_to_predict.apply(
        partial(get_score_sequence_matching, c1='journal_name',
                category=header), axis=1)
    cat_data_to_predict['pn_' + header] = cat_data_to_predict.apply(partial(get_score_sequence_matching, c1='pub_name',
                                                                            category=header), axis=1)

In [ ]:

clf = best_model.get('model')
predictions = pd.DataFrame(clf.predict(cat_data_to_predict[attributes_of_interest]))
predictions.columns = category_dummies_prefix.columns

In [ ]:

count_categories = {}
for header in predictions.columns:
    nb = predictions[header].sum()
    if nb >= 1:
        count_categories[header] = nb

In [ ]:

fig, ax = plt.subplots()
plt.bar(count_categories.keys(), count_categories.values())
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.title(f'Somme des catégories prédites par le modèle')
plt.show()

In [ ]:

# add categories predictions to their objects in table cat_data_to_predict
predictions = predictions.set_index(cat_data_to_predict.index.copy())
for header in predictions.columns:
    cat_data_to_predict[header] = predictions[header]

# Question 3: Régression-clustering
## A. Supprimer tous les attributs ayant plus de 50% de données manquantes.

On établit les attributs à éliminer selon leur taux de valeurs manquantes avec nos 3 tables originales (journal, price
et influence).
On utilise nos données déjà travaillées qui sont dans les tables cat_labelled_data et cat_data_to_predict car celles-ci
présentent toutes les données dont nous avons besoin. Comme on a des bons résultats de prédictions, on peut se 
permettre de les utiliser pour la suite du travail. C'est donc sur ces données que nous allons éliminés ces attributs.
On établit alors une nouvelle table (data) avec toutes ces données et les attributs présentant trop de valeurs 
manquantes.

In [ ]:

print(f'Attributs à éliminer de la table journal: {get_empty_attribute_to_remove(journal)}')
print(f'Attributs à éliminer de la table price: {get_empty_attribute_to_remove(price)}')
print(f'Attributs à éliminer de la table influence: {get_empty_attribute_to_remove(influence)}')

In [ ]:

cat_data_to_predict = cat_data_to_predict.drop(columns='category')
data = cat_labelled_data.append(cat_data_to_predict, sort=False)
data = data.drop(columns=['url_journal', 'influence_id', 'url_autor', 'license'])

## B. Construire un modèle pour prédire le coût actuel de publication (attribut «price») à partir des autres attributs 
(cela inclut la sélection d’attributs informatifs, le choix et le paramétrage d’un modèle de régression, le calcul du 
score du modèle, l’application du modèle pour prédire les coûts).Justifier les choix effectués.
Lister les 10 revues qui s’écartent le plus (en + ou -) de la valeur prédite.
 
 
Tout d'abord, nous décidons de ne pas utiliser les attributs que l'on a éliminé dans la question 3.A présenter un nombre conséquent de valeurs 
manquantes, que nous avons trouvés dans la question 3.A
 
L'attribut date_stamp dans price établissant la date à laquelle le prix a été mesuré semble intéressant.

On a vu question 2B que l'attribut prix n'est pas fortement corrélés aux catégories d'un journal, cependant il existait
certaines catégories présentant une certaine corrélation non négligeable. On peut reprendre alors les catégories que 
l'on a transformés en one hot à la question 2.C. Aussi, étant donné les bons résultats du modèle de classification des
catégories trouvé précédemment, on envisage d'utiliser également ces objets. 
L'information d'un journal sur son hybridicité pourrait également faire varier son prix 

Les attributs de la table influence semblent être pertinentes pour la prédiction du prix, en effet, tous les 
informations sur le nombre de citations pourraient se révéler intéressantes quant à la prédiction du prix d'un 
journal. Aussi, l'attribut informant sur le score d'influence du journal pourrait se révéler intéressant. 
Etant donné que l'attribut proj_ai_year présente toujours la même valeur, il n'est pas vraiment pertinent de le 
conserver. 

### Construction et estimation des performances du modèle

In [ ]:
# TODO: add date stamp
attributes_of_interest = [
    #     'date_stamp',
    'citation_count_sum', 'paper_count_sum', 'avg_cites_per_paper', 'proj_ai',
    'is_hybrid', 'price']
attributes_of_interest.extend(category_dummies_prefix.columns)

In [ ]:

price_labelled_data = data[attributes_of_interest]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(price_labelled_data.drop(columns='price'),
                                                    price_labelled_data['price'],
                                                    test_size=0.33, random_state=42)

regr = RandomForestRegressor(max_depth=22, n_estimators=300, n_jobs=-1)
print('-- Entrainement')
regr.fit(X_train, y_train)
train_score = regr.score(X_train, y_train)
print(f'Score d\'entraînement: {train_score}')

print('-- Test')
test_predictions = regr.predict(X_test)
test_score = regr.score(X_test, y_test)
print(f'Score de test: {test_score}')

### Application du modèle

In [ ]:

predictions = pd.DataFrame(regr.predict(price_labelled_data.drop(columns='price')))
predictions = predictions.set_index(data.index.copy())

In [ ]:

difference_pred_real = dict()
for index, p in predictions.iterrows():
    difference_pred_real[data['journal_name'][index]] = abs(p[0] - data['price'][index])

In [ ]:

print(f'Journaux dont leur prédiction du prix s\'éloigne le plus de la réalité:\n')
worst_predictions = np.array(heapq.nlargest(10, difference_pred_real, key=difference_pred_real.get))
worst_predictions_values = []
for p in worst_predictions:
    worst_predictions_values.append(difference_pred_real.get(p))
    print(f'{p} : {difference_pred_real.get(p)}')

worst_predictions = np.vstack([worst_predictions, worst_predictions_values])

## C. Construire un modèle pour grouper les revues selon le coût actuel de publication (attribut "price") et le score
d'influence (attribut "proj_ai") (cela inclut la détermination du nombre de clusters, le choix et le paramétrage d'un
modèle de clustering, l'application du modèle pour trouver les clusters). Justifier les choix.

In [ ]:

from sklearn.cluster import KMeans

attributes_of_interest = ['price', 'proj_ai']
data_for_clustering = data[attributes_of_interest]

plt.figure()
y_pred = KMeans(n_clusters=3, random_state=42).fit_predict(data_for_clustering)
plt.scatter(data_for_clustering['price'], data_for_clustering['proj_ai'], c=y_pred)
plt.show()

In [ ]:

from sklearn.cluster import OPTICS

estim = OPTICS(min_samples=20)

print(f'Entrainement et prédiction')
y_pred = estim.fit_predict(data_for_clustering)
plt.scatter(data_for_clustering['price'], data_for_clustering['proj_ai'], c=y_pred)
plt.show()
